In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import json

INPUT_DIR = "/Users/ashwins/Scripts/chatbot/"

from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, TimeDistributed
import pickle
# import nltk
import itertools
import collections

from keras.preprocessing import sequence
from keras.models import model_from_json
from keras.preprocessing import sequence
from keras.utils import np_utils

import spacy
nlp = spacy.load('en')
# from keras.utils import preprocessing
# from model.textGenModel import TextGenModel

Using TensorFlow backend.
/Library/Anaconda3/anaconda3/envs/env1/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Input, LSTM, RepeatVector, Bidirectional, Dropout
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam
from keras import objectives

In [3]:
# constant token and params for our models
START_TOKEN = "EOS"
END_TOKEN = "EOS"
UNKNOWN_TOKEN = "UNK"
PADDING_TOKEN = "PAD"

In [4]:
# vocabulary_size = 22285
sent_max_len = 20

# hidden_size = 512
hidden_size = 128
embedding_size = 128
batch_size = 64
stateful = False

In [5]:
import load_train_data
import load_vae_model
import importlib

In [ ]:
importlib.reload(load_train_data)
encoder_input_data, decoder_input_data, decoder_target_data = load_train_data.load_data()



df_lines :                                                   dialogue     l_no
0                                            They do not!    L1045
1                                             They do to!    L1044
2                                              I hope so.     L985
3                                               She okay?     L984
4                                               Let's go.     L925
5                                                     Wow     L924
6          Okay -- you're gonna need to learn how to lie.     L872
7                                                      No     L871
8       I'm kidding.  You know how sometimes you just ...     L870
9                        Like my fear of wearing pastels?     L869
10                                        The "real you".     L868
11                                       What good stuff?     L867
12      I figured you'd get to the good stuff eventually.     L866
13      Thank God!  If I had to hear one more st

In [ ]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)
print(encoder_input_data[:1])
print(decoder_input_data[:1])
print(decoder_target_data[:1])

In [ ]:
importlib.reload(load_ae_model)
autoencoder, encoder = load_ae_model.load_model()

In [ ]:
import os

# model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
opt = Adam(lr=0.001)
auto_encoder.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt)

model_json = auto_encoder.to_json()
with open("full_model_new_arch", "w") as f:
    f.write(json.dumps(json.loads((model_json))))

# Note that `decoder_target_data` needs to be one-hot encoded,
# rather than sequences of integers like `decoder_input_data`!
for i in range(4):
    if i==3:
        opt = Adam(lr=0.0001)
        auto_encoder.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt)
        
#     if i==8:
#         opt = Adam(lr=0.00001)
#         model.compile(loss='sparse_categorical_crossentropy', optimizer=opt)
        
    model_name = 'full_model_new_epoch_' + str(i) + '.h5'
    auto_encoder.save(model_name)
    print('saved model')
    
    enc_name = 'enc_model_new_epoch_' + str(i) + '.h5'
    encoder.save(enc_name)
    print('saved enc model')
    weights_path = ''.join(['enc_model_new_weights_', 
                        "base_voc_{}_epoch_{}.hdf5".format(vocabulary_size, 
                                                          i)])
    encoder.save_weights(weights_path)
    print('saved enc weights')
    # export model weights
    weights_path = ''.join(['full_model_new_weights_', 
                        "base_voc_{}_epoch_{}.hdf5".format(vocabulary_size, 
                                                          i)])
    auto_encoder.save_weights(weights_path)
    print('saved weights')
    
    auto_encoder.fit([encoder_input_data[:5000], decoder_input_data[:5000]], decoder_target_data[:5000], batch_size=batch_size, epochs=1, validation_split=0.2)
    